
*Automated Data Extraction for a Health Care Provider Using Llama Extract Using REST API - Example*

**Use Case Overview**

As part of the Health Care Provider's pr insurer’s initiative to deliver personalized plans, optimized premiums, and targeted wellness offers, the company is exploring AI-powered structured data extraction from subscriber medical records/documents. 

For the initial pilot, the insurer is focusing on records from a single partner hospital, “WellStar”. The goal is to extract legally permitted data such as subscriber names, diagnosed conditions, summaries, visit logs recurring visit details, and associated billing amounts.
________________________________________________________________________________________________________________________________________________________________

**How LIamaIndex helps for this**

Using [LlamaIndex](https://developers.llamaindex.ai/) LlamaExtract service, the insurer can convert unstructured medical PDFs into structured JSON with citations (evidence-linked text snippets) and field-level explanations, ensuring accuracy, auditability, and transparency.

Here you can find the steps to follow in setting up and executing the extraction workflow. There are are three ways by which you can extract data [Using Web UI](https://developers.llamaindex.ai/python/cloud/llamaextract/getting_started/web_ui/), [Python SDK](https://developers.llamaindex.ai/python/cloud/llamaextract/getting_started/python/), [API/REST API](https://developers.llamaindex.ai/python/cloud/llamaextract/getting_started/api/). 
_____________________________________________________________________________________________________________________________________________________________

**Rest API Overview**

This API allows insurers to create extraction agents that process hospital documents (diagnosis summaries, visit logs, invoices) from WellStar Hospital and return structured healthcare data for downstream analysis such as plan optimization, discounts, or personalized offers.


1. Sign in to [LlamaCloud](https://cloud.llamaindex.ai/)

    Access the Llama Cloud Console to manage extraction agents, run tests, and view results. All Extract-as-a-Service features are available here.

2. Generate an API Key

    You need to generate an [API Key](https://developers.llamaindex.ai/python/cloud/general/api_key) to interact with LlamaExtract, which ever method you choose to work with. Using this API key, you can access the LlamaCloud services such as [Parse](https://developers.llamaindex.ai/python/cloud/llamaparse), [Extract](https://developers.llamaindex.ai/python/cloud/llamaextract/getting_started/), and [Index](https://developers.llamaindex.ai/python/cloud/llamacloud/getting_started/). 

3. Define Your [Extraction Agent](https://developers.llamaindex.ai/python/cloud/llamaextract/features/concepts) (Schema+Settings) using Rest API.

    ***Creating a new Extraction Agent***


    Schema (What you want extracted)

    For extracting particular records in this example, the Request Example (schema) includes fields like:

    * subscriber_name
    * medical_condition
    * visit_date
    * billing_amount
    * hospital_name (filtered to WellStar)
    * recurring_visit_flag

  **Request Example**

In [ ]:
curl -X 'POST' \
  'https://api.cloud.llamaindex.ai/api/v1/extraction/extraction-agents' \
  -H 'accept: application/json' \
  -H "Authorization: Bearer $LLAMA_CLOUD_API_KEY" \
  -H 'Content-Type: application/json' \

{
  "name": "wellStar_medical_extractor",
  "data_schema": {
    "type": "object",
    "properties": {
      "subscriber_name": { "type": "string", "description": "Patient name" },
      "subscriber_id": { "type": "string", "description": "Insurance or hospital ID" },

      "medical_conditions": {
        "type": "array",
        "description": "List of diagnosed conditions",
        "items": { "type": "string" }
      },

      "visits": {
        "type": "array",
        "description": "Individual visit details",
        "items": {
          "type": "object",
          "properties": {
            "date": { "type": "string" },
            "hospital_name": { "type": "string" },
            "doctor": { "type": "string" },
            "reason": { "type": "string" }
          }
        }
      },

      "billing": {
        "type": "object",
        "properties": {
          "total_amount": { "type": "number" },
          "currency": { "type": "string" },
          "breakdown": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "item": { "type": "string" },
                "cost": { "type": "number" }
              }
            }
          }
        }
      }
    }
  },
  "config": {
    "extraction_target": "PER_DOC",
    "extraction_mode": "BALANCED"
  }
}

**Sample Response**

In [ ]:
{
  "agent_id": "agt_12ab89we90",
  "name": "wellStar_medical_extractor",
  "status": "created"
}

**Upload Document for Extraction**

    Upload a medical report PDF to extract structured data using the wellStar_medical_extractor.

**Description** 

    Upload a medical report PDF to extract structured data using the wellStar_medical_extractor.

**Request Example**

    POST /jobs
    Content-Type: multipart/form-data
    Authorization: Bearer <API_KEY>

**Form Fields**

    POST /jobs
    Content-Type: multipart/form-data
    Authorization: Bearer <API_KEY>

**Response**

In [ ]:
{
  "job_id": "job_887asdu909",
  "status": "processing",
  "estimated_time_seconds": 12
}

**Get Extraction Results**

    GET /jobs/{job_id}

In [ ]:
{
  "job_id": "job_887asdu909",
  "status": "completed",
  "results": {
    "subscriber_name": "Priya Menon",
    "subscriber_id": "WBH-44221-778",

    "medical_conditions": [
      {
        "value": "Type 2 Diabetes",
        "citations": [{ "page": 1, "text": "diagnosed with Type 2 Diabetes" }]
      },
      {
        "value": "Hypertension",
        "citations": [{ "page": 2, "text": "History: High blood pressure" }]
      }
    ],

    "visits": [
      {
        "date": "2024-06-12",
        "hospital_name": "WellBeing Hospital",
        "doctor": "Dr. Arjun Rao",
        "reason": "Quarterly diabetes monitoring",
        "citations": [{ "page": 1, "text": "visited on 12 June..." }]
      }
    ],

    "billing": {
      "total_amount": 320.00,
      "currency": "USD",
      "breakdown": [
        { "item": "Blood Test", "cost": 120.0 },
        { "item": "Consultation", "cost": 200.0 }
      ],
      "citations": [
        { "page": 3, "text": "Total Bill: $320" }
      ]
    }
  }
}


**Error Codes**

    * Code 400 - Invalid Schema or request
    * 401 - Missing/invalid API Key
    * 404 - Agent or Job not found
    * 500 - Internal extraction error